In [ ]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

In [ ]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

In [ ]:
run_dates = [(last_dow(datetime(2021, 11, 5), 4) + timedelta(days=7 * x)) for x in range(1)]
tokens = ['BTC', 'ETH', 'AVAX']
connection_string = 'sqlite:///data/data.db'
events_db = 'crypto_events_database'

In [ ]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [ ]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7)}', end='\r')
        res = pm.execute_notebook(
            'market_analysis_template.ipynb',
            f'{base_output_folder}/market_analysis_{token}_{run_date}.ipynb',
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=7)}',
                'token': token,
                'connection_string': connection_string,
                'events_db': events_db
            }
        )

In [ ]:
nb = sb.read_notebook('outputs/market_analysis_BTC_2021-11-05 00:00:00.ipynb')

In [ ]:
nb.scraps['prices'].data

In [ ]:
nbs = sb.read_notebooks('outputs')

In [ ]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'volatility': scrap['prices'].data.volatility.mean()}) 
    for book, scrap in nbs.notebook_scraps.items()
])

In [ ]:
[nbs[nb_name].reglue('summary_plot') for nb_name in nbs]